# The imshow wrapper

To simplify the use of Matplotlib's [imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html) function, **teareduce** includes a wrapper that automatically adds a colorbar. This is especially useful when analyzing astronomical images. Additionally, for wavelength-calibrated spectroscopic exposures, it allows the wavelength scale to be displayed on the horizontal axis.

In [ ]:
from astropy.io import fits
import astropy.units as u
import matplotlib.pyplot as plt
import teareduce as tea

Download the required file

In [ ]:
tea.get_cookbook_file('notebooks/aux/imshow/wcftdz_45330.fits')

We read the sample image.

In [ ]:
with fits.open('wcftdz_45330.fits') as hdul:
    header = hdul[0].header
    data = hdul[0].data

This image is wavelength calibrated.

In [ ]:
for key in ['CRPIX1', 'CRVAL1', 'CDELT1', 'CUNIT1']:
    print(f'{key}: {header[key]}')

## Simple usage

With a single instruction we can display the data array, including a colorbar and automatic labels. For that purpose we can employ the auxiliary `imshowme()` function.

In [ ]:
tea.imshowme(data)

This function is actually calling the `tea.imshow()` function described next by setting default values for the Figure and Axis objects.

## Setting the figure size

It is possible to transfer user-defined Figure and Axis instances.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
tea.imshow(fig, ax, data)
plt.tight_layout()

By default, the image is displayed using `aspect='equal'`. This can be easily modified:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
tea.imshow(fig, ax, data, aspect='auto')
plt.tight_layout()

## Displaying the wavelength 

For images calibrated in wavelength, we can specify the keywords that contain the corresponding information. This allows the figure to display a wavelength scale on the bottom horizontal axis, showing in this case the array index scale on the top horizontal axis. The units on the wavelength axis may differ from those used in the wavelength calibration.

In [ ]:
cunit1 = u.Unit(header['cunit1'])
crpix1 = header['crpix1'] * u.pix
crval1 = header['crval1'] * cunit1
cdelt1 = header['cdelt1'] * cunit1 / u.pix

print(f'{cunit1=}\n{crpix1=}\n{crval1=}\n{cdelt1=}')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
tea.imshow(fig, ax, data,
           crpix1=crpix1,
           crval1=crval1,
           cdelt1=cdelt1,
           cunit1=cunit1,
           cunitx=u.Angstrom
          )
plt.tight_layout()

## Function documentation

Note that keyword arguments not defined in the `tea.imshow()` are passed to the original Matplotlib `imshow()` function.

In [ ]:
help(tea.imshow)